# Assignment - Basic Pandas
<sup>Created by Natawut Nupairoj, Department of Computer Engineering, Chulalongkorn University</sup>

Using pandas to explore youtube trending data from GB (GBvideos.csv and GB_category_id.json) and answer the questions.

In [1]:
import pandas as pd
import numpy as np

To simplify data retrieval process on Colab, we heck if we are in the Colab environment and download data files from a shared drive and save them in folder "data".

For those using jupyter notebook on the local computer, you can read data directly assuming you save data in the folder "data".

In [2]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    !wget https://github.com/kaopanboonyuen/2110446_DataScience_2021s2/raw/main/datasets/data.tgz -O data.tgz
    !tar -xzvf data.tgz

## How many rows are there in the GBvideos.csv after removing duplications?

In [13]:
# Read data from GBvideos.csv
df = pd.read_csv('data/GBvideos.csv')

df.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,Jw1Y-zhQURU,17.14.11,John Lewis Christmas Ad 2017 - #MozTheMonster,John Lewis,26,2017-11-10T07:38:29.000Z,"christmas|""john lewis christmas""|""john lewis""|...",7224515,55681,10247,9479,https://i.ytimg.com/vi/Jw1Y-zhQURU/default.jpg,False,False,False,Click here to continue the story and make your...
1,3s1rvMFUweQ,17.14.11,Taylor Swift: …Ready for It? (Live) - SNL,Saturday Night Live,24,2017-11-12T06:24:44.000Z,"SNL|""Saturday Night Live""|""SNL Season 43""|""Epi...",1053632,25561,2294,2757,https://i.ytimg.com/vi/3s1rvMFUweQ/default.jpg,False,False,False,Musical guest Taylor Swift performs …Ready for...
2,n1WpP7iowLc,17.14.11,Eminem - Walk On Water (Audio) ft. Beyoncé,EminemVEVO,10,2017-11-10T17:00:03.000Z,"Eminem|""Walk""|""On""|""Water""|""Aftermath/Shady/In...",17158579,787420,43420,125882,https://i.ytimg.com/vi/n1WpP7iowLc/default.jpg,False,False,False,Eminem's new track Walk on Water ft. Beyoncé i...
3,PUTEiSjKwJU,17.14.11,Goals from Salford City vs Class of 92 and Fri...,Salford City Football Club,17,2017-11-13T02:30:38.000Z,"Salford City FC|""Salford City""|""Salford""|""Clas...",27833,193,12,37,https://i.ytimg.com/vi/PUTEiSjKwJU/default.jpg,False,False,False,Salford drew 4-4 against the Class of 92 and F...
4,rHwDegptbI4,17.14.11,Dashcam captures truck's near miss with child ...,Cute Girl Videos,25,2017-11-13T01:45:13.000Z,[none],9815,30,2,30,https://i.ytimg.com/vi/rHwDegptbI4/default.jpg,False,False,False,Dashcam captures truck's near miss with child ...


In [7]:
# remove duplicates
noDupDF = df.drop_duplicates()

len(noDupDF)

38745

## How many VDO's that contain at least one record (row) of "dislikes" more than "likes"?  <font color=red>DO NOT group by the data and make sure that you count only unique title!</font>

In [10]:
len(noDupDF[noDupDF['dislikes'] > noDupDF['likes']].drop_duplicates('title'))

56

## How many VDO that are trending on 22 Jan 2018 with comments more than 10,000 comments?

In [11]:
len(noDupDF[(noDupDF['trending_date'] == '18.22.01') & (noDupDF['comment_count'] > 10000)])

28

## Which date that has the minimum average number of comments per VDO?

In [14]:
noDupDF.groupby('trending_date')['comment_count'].mean().idxmin()

'17.15.11'

## Compare "Sports" and "Comady", how many days that there are more total daily views of VDO in "Sports" category than in "Comady" category?

In [28]:
# read data from GB_category_id.json
import json
with open('data/GB_category_id.json') as f:
	data = json.load(f)
categories = pd.DataFrame(data['items'])

# remove unnecessary columns
categories.drop(['etag', 'kind'], axis=1, inplace=True)

# remove unnecessaries in snippet
categories = pd.concat([categories['id'], categories['snippet'].apply(pd.Series)], axis=1)
categories.drop(['channelId'], axis=1, inplace=True)

# print categories "Sport" id
categories[(categories['title'] == 'Sports') | (categories['title'] == 'Comedy')]

,id,title,assignable
4,17,Sports,True
10,23,Comedy,True
20,34,Comedy,False


In [29]:
sports = noDupDF[noDupDF['category_id'] == 17].groupby('trending_date')['views'].sum()
comedy = noDupDF[(noDupDF['category_id'] == 23)].groupby('trending_date')['views'].sum()

len(sports[sports > comedy])

49